In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### Device configuration

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU configured")
    except Exception as e:
        print("GPU configuration error:", e)
else:
    print("No GPU found, using CPU.")

No GPU found, using CPU.


### Constants

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 5

### Load CIFAR-10 data and use training

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
subset_size = int(0.2 * x_train.shape[0])
x_train, y_train = x_train[:subset_size], y_train[:subset_size]

### Data Preprocessing & Pipeline

In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    label = tf.one_hot(tf.squeeze(label), NUM_CLASSES)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                   .shuffle(1024)\
                   .batch(BATCH_SIZE)\
                   .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                 .batch(BATCH_SIZE)\
                 .prefetch(tf.data.AUTOTUNE)

### Load the pre-trained MobileNet (without top layers) and freeze it

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

### Build the model

In [ ]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = models.Model(inputs, outputs)

### Display model architecture

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                10250     
                                                                 
Total params: 3,239,114
Trainable params: 10,250
Non-trainable params: 3,228,864
_________________________________________________________________


### Callbacks for robust training

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
    ModelCheckpoint("baseline_model.h5", monitor='val_loss', save_best_only=True)
]

### Compile and train the model

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("\nStarting baseline training...\n")
history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=callbacks, verbose=1)


Starting baseline training...

Epoch 1/5
313/313 [==============================] - 274s 875ms/step - loss: 0.9051 - accuracy: 0.6947 - val_loss: 0.6111 - val_accuracy: 0.7942
Epoch 2/5
313/313 [==============================] - 265s 847ms/step - loss: 0.5070 - accuracy: 0.8330 - val_loss: 0.5210 - val_accuracy: 0.8216
Epoch 3/5
313/313 [==============================] - 267s 854ms/step - loss: 0.4243 - accuracy: 0.8580 - val_loss: 0.4811 - val_accuracy: 0.8372
Epoch 4/5
313/313 [==============================] - 277s 886ms/step - loss: 0.3827 - accuracy: 0.8733 - val_loss: 0.4614 - val_accuracy: 0.8416
Epoch 5/5
313/313 [==============================] - 268s 856ms/step - loss: 0.3438 - accuracy: 0.8862 - val_loss: 0.4622 - val_accuracy: 0.8423


### Evaluate and final accuracy

In [ ]:
loss, acc = model.evaluate(test_ds, verbose=0)
print(f"\nFinal Test Accuracy: {acc*100:.2f}%")


Final Test Accuracy: 84.23%
